In [1]:
%load_ext autoreload
%autoreload

import io
import os
import sys
import django
import glob
import json
import re
import requests
from IPython.core.debugger import set_trace
from pathlib import Path
from django.core.files.base import ContentFile
from rest_framework import serializers
from tqdm.notebook import tqdm
from google.cloud import vision
from google.cloud import storage
from PIL import Image, ImageSequence
from gensim.models import Doc2Vec
from IPython.core.debugger import set_trace
import IPython
import collections
from googletrans import Translator
from google.cloud import translate_v2 as translate

parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)
django.setup()

import getch.models as m
from django.db.models import F, Q, Sum, Count, Case, When, IntegerField, Value
from django.apps import apps
from django.conf import settings
from django_currentuser.middleware import get_current_user
from datetime import datetime, timedelta
import time
import pandas as pd

In [2]:
# trans = m.Transaction.objects.select_related('sender','receiver'); trans
trans = m.Transaction.objects.select_related(
    'sender',
    'sender__boo','sender__boo__user',
    'sender__raffle','sender__raffle__item',
    'sender__shoptem','sender__shoptem__item',
    'sender__coffeecoupon','sender__coffeecoupon__item',
    'sender__support','sender__support__brand',
    'receiver',
    'receiver__boo','receiver__boo__user',
    'receiver__raffle','receiver__raffle__item',
    'receiver__support','receiver__support__brand',
    'receiver__shoptem','receiver__shoptem__item',
    'receiver__coffeecoupon','receiver__coffeecoupon__item',
); trans

<QuerySet [<Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>, <Transaction: quantlab | quantlab@kakao.com > POSTTO | 포스토 | 2021-08-06 16:21:02>, <Transaction: quantlab | quantlab@kakao.com > STARBUCKS LAMP BK - 0 | 2021-08-07 20:37:53>, <Transaction: quantlab | quantlab@kakao.com > TAW&TOE ZEROVITY™ Flip Flop, SIZE M - 38000 | 2021-08-07 23:34:16>, <Transaction: quantlab | quantlab@kakao.com > KINTO TUMBLER 350ML - 39000 | 2021-08-07 20:38:17>, <Transaction: quantlab | quantlab@kakao.com > JOHNBER | 존버 스포츠 | 2021-08-10 18:02:20>, <Transaction: quantlab | quantlab@kakao.com > BADBOY | 배드보이 | 2021-08-08 14:43:50>,

# 모든 트랜젝션 가져오기

In [ ]:
trans_list = []
for _t in trans:
    tr = {}
    tr_type = _t.get_type_display()
    tr['user'] = str(_t.sender) if tr_type in ['래플','후원'] else str(_t.receiver)
    tr['when'] = str(_t.when.date())
    tr['type'] = tr_type
    tr['amount'] = -_t.amount if tr_type in ['래플','후원'] else _t.amount
    trans_list.append(tr)

# 트랜젝션 횟수 시계열

In [6]:
counts = pd.DataFrame(trans_list).groupby(['user','type','when']).count().unstack().fillna(0)
counts = counts.groupby('type').sum()[counts.columns[-29:-1]].drop(['첫방문환영','쇼핑','커피쿠폰','테스트 INFLOW']).astype('int64')
#counts.style.set_table_attributes('style="font-size: 10px"')
counts.T

type               관심스타일입력  관심아이템입력  기본정보입력  래플  리서치참여  매일밸런스게임  옷장넣기  출첵게임  \
       when                                                                   
amount 2021-09-13        0        0       3   0     32      402     0    30   
       2021-09-14        1        0       0   0      6      243     0    18   
       2021-09-15        0        0       2   3     23      499     0    19   
       2021-09-16        1        1       3   2     80      712     2    27   
       2021-09-17        0        0       2  12     18     1000     3    37   
       2021-09-18        1        1      10   7     58      639     4    21   
       2021-09-19        1        1       1  13     30      751     6    15   
       2021-09-20        0        0       0   0      8      819     1    16   
       2021-09-21        0        0       2   1     33      775     1    24   
       2021-09-22        0        0       3   2     19      886     3    25   
       2021-09-23        0        0       2   1     28     1041     4    28   
       2021-09-24        2        2       4   5     42      988     2    21   
       2021-09-25        0        0       2   4     37     1167     2    26   
       2021-09-26        2        2       4   4     41     1118     0    32   
       2021-09-27        2        1       4   2     36      853     1    20   
       2021-09-28        1        1       3   3     40      918     3    41   
       2021-09-29        1        1       3   1     42     1080     2    33   
       2021-09-30        2        2       7   3    119     2048     1    52   
       2021-10-01        1        1       6   9     70     1696     0    61   
       2021-10-02        0        0       1   5     16     1683     1    31   
       2021-10-03        1        1       0   2     23     1746     2    25   
       2021-10-04        0        0       0   1     19     1616     5    30   
       2021-10-05        3        3       5   0     16     1809     0    34   
       2021-10-06        0        0       1   8     28     1572     3    39   
       2021-10-07        1        1       1   7     27     1602     0    28   
       2021-10-08        0        0       1  12      7     1624     4    34   
       2021-10-09        1        1       1   6     21     1415     0    41   
       2021-10-10        0        0       2  14     13     1847     1    34   

type               후원  
       when            
amount 2021-09-13   0  
       2021-09-14   0  
       2021-09-15   5  
       2021-09-16   6  
       2021-09-17   3  
       2021-09-18   0  
       2021-09-19   0  
       2021-09-20   0  
       2021-09-21   0  
       2021-09-22   0  
       2021-09-23   0  
       2021-09-24   0  
       2021-09-25   1  
       2021-09-26   0  
       2021-09-27   0  
       2021-09-28   1  
       2021-09-29   1  
       2021-09-30   0  
       2021-10-01   0  
       2021-10-02   0  
       2021-10-03   0  
       2021-10-04   0  
       2021-10-05   0  
       2021-10-06   0  
       2021-10-07   0  
       2021-10-08   0  
       2021-10-09   0  
       2021-10-10   0

# 트랜젝션 금액 시계열

In [7]:
amounts = pd.DataFrame(trans_list).groupby(['user','type','when']).sum().unstack().fillna(0)
amounts = amounts.groupby('type').sum()[amounts.columns[-29:-1]].drop(['첫방문환영','쇼핑','커피쿠폰','테스트 INFLOW']).astype('int64')
#amounts.style.set_table_attributes('style="font-size: 10px"')
amounts.T

type               관심스타일입력  관심아이템입력  기본정보입력    래플  리서치참여  매일밸런스게임  옷장넣기  출첵게임  \
       when                                                                     
amount 2021-09-13        0        0     300     0   3640      400     0   240   
       2021-09-14       50        0       0     0    180      243     0   140   
       2021-09-15        0        0     200  -288   2020      475     0   130   
       2021-09-16       50       50     300  -143   8010      700     2   230   
       2021-09-17        0        0     200 -1050   2140      900     3   230   
       2021-09-18       50       50     400  -641   6270      635     4   190   
       2021-09-19       50       50     100 -1398   2770      685     6   110   
       2021-09-20        0        0       0     0    510      801     1   140   
       2021-09-21        0        0     200   -77   3670      735     1   180   
       2021-09-22        0        0     300  -222   1380      846     3   210   
       2021-09-23        0        0     200   -77   2980      973     4   200   
       2021-09-24      100      100     400  -464   4440      976     2   170   
       2021-09-25        0        0     200  -398   3770     1135     2   200   
       2021-09-26      100      100     400  -410   3890     1098     0   240   
       2021-09-27      100       50     400  -210   3170      831     1   200   
       2021-09-28       50       50     300  -288   4550      808     3   310   
       2021-09-29       50       50     300   -77   4790     1060     2   290   
       2021-09-30      100      100     700  -265  13060     1662     1   460   
       2021-10-01       50       50     600  -875   6640     1442     0   490   
       2021-10-02        0        0     100  -500   1460     1361     1   250   
       2021-10-03       50       50       0  -222   2500     1434     2   230   
       2021-10-04        0        0       0  -111   2420     1364     5   260   
       2021-10-05      150      150     500     0   1460     1649     0   340   
       2021-10-06        0        0     100  -811   3520     1472     3   330   
       2021-10-07       50       50     100  -733   2370     1584     0   260   
       2021-10-08        0        0     100 -1321    480     1508     4   300   
       2021-10-09       50       50     100  -666   2460     1367     0   310   
       2021-10-10        0        0       0 -1532   2550     1685     1   300   

type                후원  
       when             
amount 2021-09-13    0  
       2021-09-14    0  
       2021-09-15 -500  
       2021-09-16 -600  
       2021-09-17 -300  
       2021-09-18    0  
       2021-09-19    0  
       2021-09-20    0  
       2021-09-21    0  
       2021-09-22    0  
       2021-09-23    0  
       2021-09-24    0  
       2021-09-25 -100  
       2021-09-26    0  
       2021-09-27    0  
       2021-09-28 -100  
       2021-09-29 -100  
       2021-09-30    0  
       2021-10-01    0  
       2021-10-02    0  
       2021-10-03    0  
       2021-10-04    0  
       2021-10-05    0  
       2021-10-06    0  
       2021-10-07    0  
       2021-10-08    0  
       2021-10-09    0  
       2021-10-10    0

# 가입자수 추이

In [4]:
qrs = m.User.objects.filter(date_joined__gte='2021-09-13').values('date_joined__date').annotate(count=Count('date_joined__date'))

In [5]:
pd.DataFrame(qrs).set_index('date_joined__date').sort_index(ascending=True).iloc[:-1].sum()

count    238
dtype: int64

# 서베이별 참여자수

In [2]:
_boos = m.Boo.objects.filter(answers__has_key='12')

In [3]:
_boos[10].answers

{'4': {'11': [1],
  '12': [0],
  '13': [1],
  '14': [0],
  '15': [1],
  '16': [1],
  '17': [0],
  '18': [0],
  '19': [0],
  '20': [1],
  '21': [1],
  '22': [0],
  '23': [0],
  '24': [0],
  '25': [0],
  'finished': True},
 '5': {'26': [1],
  '27': [1],
  '28': [1],
  '29': [0],
  '30': [0],
  '31': [1],
  '32': [0, 1],
  '33': [0],
  '34': [1],
  '35': [1],
  '36': [1],
  '37': [0],
  '38': [0],
  '39': [0],
  '40': [0],
  '41': [1],
  'finished': True},
 '6': {'42': [1],
  '43': [0],
  '44': [3],
  '45': [0],
  '46': [0],
  '115': [1],
  '117': [1],
  '118': [1],
  '119': [2],
  '120': [1],
  '121': [5],
  '122': [1, 2],
  '123': [0],
  '124': [1],
  'finished': True},
 '7': {'47': [1],
  '48': [1],
  '49': [0],
  '50': [0],
  '51': [1],
  '171': [0],
  '173': [0],
  '174': [0],
  '175': [1],
  '177': [0],
  '178': [1],
  '179': [0],
  '181': [1],
  '182': [1],
  '183': [0],
  'finished': True},
 '8': {'52': [1],
  '53': [1],
  '54': [1],
  '55': [1],
  '56': [1],
  '58': [1],
  '59': 

In [ ]:
배드보이 4
터치그라운드 5
존버 6
포스토 7
브라스보트 8

비오는날 취향분석 9
슬기로운 캠필생활 10
나에게 딱 맞는 주말 베스트 코디 12
내 휴가 스타일 14
핫서머에 입기 좋은 핫팬츠 취향조사 15
구두취향 본격탐구 17
내 백 취향 18
요즘 막 당신의 관심사 19
비오는날 샌들 취향조사 20
어느 브랜드 좋아해 22
올가을 트랜드 조사 신발편 23
가을 유니폼 후디 맨투맨 고르기 24
부츠 이렇게 신어야지 25
이번 fw 뭐 신지 26


In [11]:
m.Boo.objects.filter(answers__has_key='4').count()

129

In [2]:
pd.DataFrame({
#     '배드보이': [m.Boo.objects.filter(answers__has_key='4').count()],
#     '터치그라운드': [m.Boo.objects.filter(answers__has_key='5').count()],
#     '존버': [m.Boo.objects.filter(answers__has_key='6').count()],
#     '포스토': [m.Boo.objects.filter(answers__has_key='7').count()],
#     '브라스보트': [m.Boo.objects.filter(answers__has_key='8').count()],
    
#     '비오는날 취향분석': [m.Boo.objects.filter(answers__has_key='9').count()],
#     '슬기로운 캠핑생활': [m.Boo.objects.filter(answers__has_key='10').count()],
#     '주말 베스트 코디': [m.Boo.objects.filter(answers__has_key='12').count()],
#     '내 휴가 스타일': [m.Boo.objects.filter(answers__has_key='14').count()],
#     '핫팬츠 취향조사': [m.Boo.objects.filter(answers__has_key='15').count()],
#     '구두취향 본격탐구': [m.Boo.objects.filter(answers__has_key='17').count()],
#     '내 백 취향': [m.Boo.objects.filter(answers__has_key='18').count()],
#     '요즘 막 당신의 관심사': [m.Boo.objects.filter(answers__has_key='19').count()],
#     '샌들 취향조사': [m.Boo.objects.filter(answers__has_key='20').count()],
#     '어느 브랜드 좋아해': [m.Boo.objects.filter(answers__has_key='22').count()],
#     '올가을 트랜드 조사 신발편': [m.Boo.objects.filter(answers__has_key='23').count()],
#     '가을 유니폼 후디 맨투맨 고르기': [m.Boo.objects.filter(answers__has_key='24').count()],
#     '부츠 이렇게 신어야지': [m.Boo.objects.filter(answers__has_key='25').count()],
    '[아디다스A]우린 골퍼잖아': [m.Boo.objects.filter(answers__has_key='32').count()],
    '[아디다스B]나이스샷, 골프웨어 브랜딩': [m.Boo.objects.filter(answers__has_key='33').count()],
    '[배드보이A]배드보이를 카운터해주세요': [m.Boo.objects.filter(answers__has_key='34').count()],
    '[배드보이B]운동을 즐기는 멋쟁이들': [m.Boo.objects.filter(answers__has_key='35').count()],
    '[닥터마틴A]클릭,당신만의 라이프스타일': [m.Boo.objects.filter(answers__has_key='30').count()],
    '[닥터마틴B]패션에도 비건이': [m.Boo.objects.filter(answers__has_key='31').count()],
}).T

,0
[아디다스A]우린 골퍼잖아,37
"[아디다스B]나이스샷, 골프웨어 브랜딩",36
[배드보이A]배드보이를 카운터해주세요,18
[배드보이B]운동을 즐기는 멋쟁이들,43
"[닥터마틴A]클릭,당신만의 라이프스타일",47
[닥터마틴B]패션에도 비건이,26


In [12]:
pd.DataFrame({
    '[아디다스A]우린 골퍼잖아': [m.Boo.objects.filter(answers__has_key='32').count()],
    '[아디다스B]나이스샷, 골프웨어 브랜딩': [m.Boo.objects.filter(answers__has_key='33').count()],
    '[배드보이A]배드보이를 카운터해주세요': [m.Boo.objects.filter(answers__has_key='34').count()],
    '[배드보이B]운동을 즐기는 멋쟁이들': [m.Boo.objects.filter(answers__has_key='35').count()],
    '[닥터마틴A]클릭,당신만의 라이프스타일': [m.Boo.objects.filter(answers__has_key='30').count()],
    '[닥터마틴B]패션에도 비건이': [m.Boo.objects.filter(answers__has_key='31').count()],
}).T

,0
[아디다스A]우린 골퍼잖아,46
"[아디다스B]나이스샷, 골프웨어 브랜딩",48
[배드보이A]배드보이를 카운터해주세요,26
[배드보이B]운동을 즐기는 멋쟁이들,59
"[닥터마틴A]클릭,당신만의 라이프스타일",56
[닥터마틴B]패션에도 비건이,35


In [43]:
suv_id = '31'
answers = m.Boo.objects.filter(answers__has_key=suv_id).values('id','nick','answers'); answers

<QuerySet [{'id': 933, 'nick': 'hsk1230', 'answers': {'4': {'11': [0], '12': [0], '13': [1], '14': [1], '15': [0], '16': [0], '17': [7], '18': [0], '19': [0], '20': [0], '21': [0], '22': [1], '23': [0], '24': [0], '25': [0], 'finished': True}, '5': {'26': [1], '27': [0], '28': [1], '29': [0], '30': [0], '31': [1], '32': [10], '33': [1], '34': [1], '35': [1], '36': [0], '37': [0], '38': [1], '39': [1], '40': [4], '41': [1], 'finished': True}, '6': {'42': [0], '43': [0], '44': [3], '45': [1], '46': [0], '115': [1], '116': ['아이다'], '117': [0], '118': [0], '119': [4], '120': [1], '121': [2, 4, 5, 0, 3], '122': [1, 4, 5], '123': [0], '124': [0], 'finished': True}, '7': {'47': [1], '48': [1], '49': [1], '50': [0], '51': [1], '171': [1], '173': [0], '174': [0], '175': [1], '177': [1], '178': [1], '179': [1], '181': [0], '182': [0], '183': [0], 'finished': True}, '8': {'52': [0], '53': [1], '54': [1], '55': [3], '56': [1], '57': ['프라다'], '58': [0], '59': [0], '60': [1], '61': [0], '62': [0], '

In [44]:
answers_4 = [{'id':o['id'], 'nick':o['nick'], 'answers':o['answers'][suv_id]} for o in answers]; answers_4

[{'id': 933,
  'nick': 'hsk1230',
  'answers': {'297': [0],
   '299': [2, 0],
   '300': [0],
   '301': [0],
   '302': [2],
   '303': [0],
   '304': [0],
   '305': [1],
   '306': [1],
   '307': [1],
   'finished': True}},
 {'id': 1000,
  'nick': 'forjin3',
  'answers': {'297': [0],
   '299': [0],
   '300': [0],
   '301': [1],
   '302': [4],
   '303': [4],
   '304': [1],
   '305': [1],
   '306': [1],
   '307': [1],
   '356': [5],
   'finished': True}},
 {'id': 970,
  'nick': 'd0hwan',
  'answers': {'297': [0],
   '299': [1, 0],
   '300': [0],
   '301': [1],
   '302': [2, 1],
   '303': [2],
   '304': [1],
   '305': [1],
   '306': [1],
   '307': [0],
   'finished': True}},
 {'id': 937,
  'nick': 'gunhye100',
  'answers': {'297': [0],
   '299': [0],
   '300': [1],
   '301': [1],
   '302': [4],
   '303': [4],
   '304': [1],
   '305': [0],
   '306': [1],
   '307': [1],
   '356': [5],
   'finished': True}},
 {'id': 1082,
  'nick': 'didtks26',
  'answers': {'297': [1],
   '299': [4],
   '300': 

In [45]:
df = pd.DataFrame(answers_4); df

,id,nick,answers
0,933,hsk1230,"{'297': [0], '299': [2, 0], '300': [0], '301':..."
1,1000,forjin3,"{'297': [0], '299': [0], '300': [0], '301': [1..."
2,970,d0hwan,"{'297': [0], '299': [1, 0], '300': [0], '301':..."
3,937,gunhye100,"{'297': [0], '299': [0], '300': [1], '301': [1..."
4,1082,didtks26,"{'297': [1], '299': [4], '300': [1], '301': [1..."
5,425,Seoul_domestic_m,"{'297': [0], '299': [4], '300': [1], '301': [0..."
6,940,kimjjmm,"{'297': [0], '299': [4], '300': [1], '301': [1..."
7,643,giseok1929,"{'297': [1], '299': [0], '300': [0], '301': [1..."
8,105,leesuyong,"{'297': [0], '299': [0, 3, 1], '300': [1], '30..."
9,797,kojedong287,"{'297': [1], '299': [4], '300': [1], '301': [0..."


In [46]:
def conv(x):
    try:
        if type(x)==bool:
            return x
        
        str_x = [str(n) for n in x]
        return ', '.join(str_x)
    
    except:
        return ''
    
df2 = pd.DataFrame(list(df['answers'].values)).applymap(conv); df2

,297,299,300,301,302,303,304,305,306,307,finished,356
0,0,"2, 0",0,0,2,0,0,1,1,1,True,
1,0,0,0,1,4,4,1,1,1,1,True,5
2,0,"1, 0",0,1,"2, 1",2,1,1,1,0,True,
3,0,0,1,1,4,4,1,0,1,1,True,5
4,1,4,1,1,4,4,1,0,1,1,True,
5,0,4,1,0,4,4,1,1,1,1,True,3
6,0,4,1,1,4,4,1,0,0,1,True,
7,1,0,0,1,4,"0, 4",1,1,0,0,True,
8,0,"0, 3, 1",1,0,"2, 1",0,1,0,1,0,True,5
9,1,4,1,0,4,4,1,1,1,,True,


In [47]:
df3 = pd.concat([df, df2], axis=1).drop(['answers'], axis=1); df3

,id,nick,297,299,300,301,302,303,304,305,306,307,finished,356
0,933,hsk1230,0,"2, 0",0,0,2,0,0,1,1,1,True,
1,1000,forjin3,0,0,0,1,4,4,1,1,1,1,True,5
2,970,d0hwan,0,"1, 0",0,1,"2, 1",2,1,1,1,0,True,
3,937,gunhye100,0,0,1,1,4,4,1,0,1,1,True,5
4,1082,didtks26,1,4,1,1,4,4,1,0,1,1,True,
5,425,Seoul_domestic_m,0,4,1,0,4,4,1,1,1,1,True,3
6,940,kimjjmm,0,4,1,1,4,4,1,0,0,1,True,
7,643,giseok1929,1,0,0,1,4,"0, 4",1,1,0,0,True,
8,105,leesuyong,0,"0, 3, 1",1,0,"2, 1",0,1,0,1,0,True,5
9,797,kojedong287,1,4,1,0,4,4,1,1,1,,True,


In [48]:
df3.to_excel('drmartens_b_result_final.xlsx')

In [60]:
items = list(df3.columns)
items.remove('id')
items.remove('nick')
items.remove('finished')
items

['297', '299', '300', '301', '302', '303', '304', '305', '306', '307', '356']

In [61]:
df4 = pd.DataFrame(m.ResearchItem.objects.filter(pk__in=items).values('id','type','text')); df4

,id,type,text
0,299,MC,가치소비를 하는 이유는?
1,302,MC,구매하신 비건 제품은 무엇인가요?
2,303,MC,구매하시게 된 경로는 무엇인가요?
3,307,OX,닥터마틴에서 비건슈즈를 출시한다면 구매의사가 있으신가요?
4,297,OX,가치소비에 대해 아시나요??
5,300,OX,비건제품을 구매하신 적이 있나요?
6,306,OX,닥터마틴의 비건슈즈를 아시나요?
7,304,OX,비건소비를 패션에도 실천하신 적이 있으신가요?
8,301,AB,비건에 관심있는 당신의 성별은?
9,305,AB,비건슈즈가 출시된다면?


In [62]:
df4.to_excel('drmartens_b_items.xlsx')

# 포인트 현황

In [3]:
memb_list = ['quantlab@kakao.com',
'gem763@gmail.com',
'sejin76@nate.com',
'magrittelim@gmail.com',
'magrittelim@naver.com',
'smilekhm@gmail.com',
'hmkoo@kakao.com',
'giseok1929@daum.net',
'gentlebaek89@gmail.com',
'leesuyong@hotmail.com']

In [4]:
wallet_stat = []
for w in tqdm(m.Wallet.objects.all()[:]):
    if w.whose_type == 'boo':
        wallet_stat.append({
            'whose': str(w.whose), 
            'n_transaction': w.n_transaction, 
            'inflow': w.inflow, 
            'outflow': w.outflow, 
            'amount': w.amount
        })

  0%|          | 0/549 [00:00<?, ?it/s]

In [5]:
wallet_stat

[{'whose': 'quantlab | quantlab@kakao.com',
  'n_transaction': 1176,
  'inflow': 56877,
  'outflow': 10785,
  'amount': 46092},
 {'whose': '모이버 | moiber@moiber.com',
  'n_transaction': 87488,
  'inflow': 11,
  'outflow': 778744,
  'amount': -778733},
 {'whose': '홈마 | quantlab@kakao.com',
  'n_transaction': 0,
  'inflow': 0,
  'outflow': 0,
  'amount': 0},
 {'whose': 'Seoul_domestic_m | magrittelim@gmail.com',
  'n_transaction': 2012,
  'inflow': 108412,
  'outflow': 23710,
  'amount': 84702},
 {'whose': 'minimal_ootd | magrittelim@gmail.com',
  'n_transaction': 0,
  'inflow': 0,
  'outflow': 0,
  'amount': 0},
 {'whose': 'giseok1929 | giseok1929@daum.net',
  'n_transaction': 444,
  'inflow': 57184,
  'outflow': 2060,
  'amount': 55124},
 {'whose': 'gentlebaek89 | gentlebaek89@gmail.com',
  'n_transaction': 1607,
  'inflow': 57453,
  'outflow': 0,
  'amount': 57453},
 {'whose': 'smilekhm | smilekhm@gmail.com',
  'n_transaction': 238,
  'inflow': 53086,
  'outflow': 5165,
  'amount': 479

In [6]:
df = pd.DataFrame(wallet_stat).sort_values(by=['amount'], ascending=False); df

,whose,n_transaction,inflow,outflow,amount
3,Seoul_domestic_m | magrittelim@gmail.com,2012,108412,23710,84702
6,gentlebaek89 | gentlebaek89@gmail.com,1607,57453,0,57453
5,giseok1929 | giseok1929@daum.net,444,57184,2060,55124
7,smilekhm | smilekhm@gmail.com,238,53086,5165,47921
0,quantlab | quantlab@kakao.com,1176,56877,10785,46092
...,...,...,...,...,...
456,windtk | windtk@naver.com,0,0,0,0
104,sua77331 | sua77331@gmail.com,0,0,0,0
33,Celeb_snaps_s | magrittelim@gmail.com,0,0,0,0
144,uksaar | uksaar@naver.com,0,0,0,0


In [9]:
df.iloc[6:].iloc[:20]

,whose,n_transaction,inflow,outflow,amount
326,rinahuh | rinahuh@naver.com,345,9880,0,9880
172,skyehdgnl | skyehdgnl@naver.com,4926,9784,0,9784
143,zawon22 | zawon22@naver.com,4823,8873,0,8873
276,hsk1230 | hsk1230@hanmail.net,3948,8857,0,8857
280,gunhye100 | gunhye100@hanmail.net,4055,8300,0,8300
249,네리 | hyerenia@nate.com,125,8277,0,8277
343,forjin3 | forjin3@naver.com,3470,7801,0,7801
308,godtls14 | godtls14@nate.com,4080,7789,0,7789
135,kojedong287 | kojedong287@naver.com,5513,10080,4100,5980
267,sooho28956 | sooho28956@naver.com,1375,5860,0,5860


# 래플참여 현황

In [109]:
raffle_result = trans.filter(type=101).values('sender__boo__user__email', 'receiver__raffle__item__name', 'amount')

In [122]:
df = pd.DataFrame(raffle_result)
df.columns = ['유저','래플','참여']
df = df[~df['유저'].isin(memb_list)]

In [123]:
df.groupby(by=['래플','유저']).count()

참여
래플                   유저                       
KINTO TUMBLER 350ML  dydwk231@naver.com      1
                     fredasd0120@gmail.com   1
                     heetae002@naver.com     7
                     jueun0618@gmail.com     1
                     jylee118218@naver.com   1
...                                         ..
오징어게임 프론트맨 가면        london1920h@naver.com   1
                     ssopie@gmail.com        1
                     thfk3333@gmail.com      2
존버 대용량 철인3종 수영가방 메시백 kimjjmm@hanmail.net     1
                     liokjkr@nate.com        1

[117 rows x 1 columns]

In [114]:
df.groupby(by=['래플','유저']).count().to_excel('raffle_stat_20211025.xlsx')

In [124]:
df.groupby(by=['래플','유저']).count().loc['오징어게임 프론트맨 가면'].sum()

참여    11
dtype: int64

In [15]:
wallet = m.Boo.objects.get(user__email='youngee202@naver.com').wallet; wallet

<Wallet: youngee202 | youngee202@naver.com>

In [19]:
for t in m.Transaction.objects.filter(receiver__boo__user__email='youngee202@naver.com', type__in=[0,1]).order_by('-id'):
    #t.receiver = wallet
    #t.save()
    #break
    print(t.when, t.get_type_display())

2021-10-26 16:11:20.507327 매일밸런스게임
2021-10-26 16:11:17.449442 매일밸런스게임
2021-10-26 16:11:13.173979 매일밸런스게임
2021-10-26 16:11:03.283585 매일밸런스게임
2021-10-26 16:10:58.561916 매일밸런스게임
2021-10-26 16:10:56.880985 매일밸런스게임
2021-10-26 16:10:31.394980 매일밸런스게임
2021-10-26 16:10:24.252203 매일밸런스게임
2021-10-26 16:10:19.758004 매일밸런스게임
2021-10-26 16:10:14.385890 매일밸런스게임
2021-10-26 16:10:11.958464 매일밸런스게임
2021-10-26 16:10:07.489429 매일밸런스게임
2021-10-26 16:10:01.406676 매일밸런스게임
2021-10-26 16:09:54.376849 매일밸런스게임
2021-10-26 16:09:51.408353 매일밸런스게임
2021-10-26 16:09:42.526526 매일밸런스게임
2021-10-26 16:00:21.869166 출첵게임
2021-10-26 11:18:28.072287 출첵게임
2021-10-25 15:12:09.756881 출첵게임
2021-10-24 11:07:22.309387 출첵게임
2021-10-21 16:04:14.268578 출첵게임
2021-10-14 11:34:28.577910 출첵게임
2021-10-12 12:38:56.434321 매일밸런스게임
2021-10-12 12:37:42.962407 출첵게임
2021-10-09 19:46:47.661254 매일밸런스게임
2021-10-09 19:46:42.060080 매일밸런스게임
2021-10-09 19:46:30.154403 출첵게임
2021-09-26 13:29:26.204014 매일밸런스게임
2021-09-26 12:23:27.869999 매일밸런스게임
2021-09-2